# 初始化設定

## 將程式下載到colab環境

In [1]:
!git clone https://github.com/SHI-Labs/FcF-Inpainting.git
%cd FcF-Inpainting

Cloning into 'FcF-Inpainting'...
remote: Enumerating objects: 307, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 307 (delta 33), reused 26 (delta 26), pack-reused 257 (from 1)
Receiving objects: 100% (307/307), 38.52 MiB | 11.59 MiB/s, done.
Resolving deltas: 100% (75/75), done.
/content/FcF-Inpainting


## 上傳資料集

In [11]:
#將資料集複製到執行目錄下(由於dataset.py的載入方法基於執行目錄的相對路徑，因此在不更動程式的情況下這樣處理)
import shutil

In [13]:
# 複製data資料集
source_path = "/content/drive/MyDrive/中興_深度學習/HW4/data"  # 請替換成你的data資料集的絕對路徑
target_path = "/content/FcF-Inpainting/mydata"        # Colab 中的執行目錄
shutil.copytree(source_path, target_path)
print(f"資料夾已從 {source_path} 複製到 {target_path}")


資料夾已從 /content/drive/MyDrive/中興_深度學習/HW4/data 複製到 /content/FcF-Inpainting/mydata


In [ ]:
#將data資料集複製到執行目錄下
source_path = "/content/drive/MyDrive/中興_深度學習/HW4/test"  # 請替換成你的test資料集的絕對路徑
target_path = "/content/FcF-Inpainting/mytest"        # Colab 中的執行目錄
shutil.copytree(source_path, target_path)
print(f"資料夾已從 {source_path} 複製到 {target_path}")

資料夾已從 /content/drive/MyDrive/中興_深度學習/HW4/test 複製到 /content/Pytorch-MADF/test


## 設定環境與參數

In [2]:
!pip3 install -r requirements.txt
!export TORCH_HOME=$(pwd) && export PYTHONPATH=.

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.5/271.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.7 MB/s eta 0:00:00
  Created wheel for etaprogress: filename=etaprogress-1.1.1-py3-none-any.whl size=15611 sha256=290184975a6e4091efa32eff32e6e4a9946811248c8b3a8b4b6e4cf1e8612eec
  Stored in directory: /root/.cache/pip/wheels/6d/58/ae/cb84257569b9d1ab28a2c1a5bd3bb13c00ae84a1eb04170078
Successfully built etaprogress
  Attempting uninstall: imageio-ffmpeg
    Found existing installation: imageio-ffmpeg 0.5.1
    Uninstalling imageio-ffmpeg-0.5.1:
      Successfully uninstalled imageio-ffmpeg-0.5.1


In [3]:
!mkdir -p ade20k/ade20k-resnet50dilated-ppm_deepsup/
!wget -P ade20k/ade20k-resnet50dilated-ppm_deepsup/ http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth

--2024-12-29 07:09:10--  http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth
Resolving sceneparsing.csail.mit.edu (sceneparsing.csail.mit.edu)... 128.30.100.223
Connecting to sceneparsing.csail.mit.edu (sceneparsing.csail.mit.edu)|128.30.100.223|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95015426 (91M)
Saving to: ‘ade20k/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth’

encoder_epoch_20.pt 100%[===================>]  90.61M  3.24MB/s    in 32s     

2024-12-29 07:09:43 (2.81 MB/s) - ‘ade20k/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth’ saved [95015426/95015426]



In [18]:
def remove_bom(file_path):
    """Remove BOM (Byte Order Mark) from a file if present."""
    with open(file_path, "rb") as file:
        content = file.read()

    # Check if BOM exists and remove it
    if content.startswith(b'\xef\xbb\xbf'):
        content = content[3:]

    with open(file_path, "wb") as file:
        file.write(content)

    print(f"BOM removed from {file_path}")

# Example usage
remove_bom("training/training_loop.py")
remove_bom("training/data/dataset.py")


BOM removed from training/training_loop.py
BOM removed from training/data/dataset.py


In [14]:
IMG_DATA = '/content/FcF-Inpainting/mydata/place2_train'
VAL_IMG_DATA = '/content/FcF-Inpainting/mydata/place2_val'
OUT_DIR = '/content/FcF-Inpainting/myresult'

In [19]:
#!python train.py --outdir=training-runs-inp --img_data=$IMG_DATA --gpus 8 --kimg 25000 --gamma 10 --aug 'noaug' --metrics True --eval_img_data $VAL_IMG_DATA --batch 128

!python train.py --outdir=training-runs-inp --img_data=$IMG_DATA --gpus 1 --kimg 1 --gamma 10 --aug 'noaug' --metrics True --eval_img_data $VAL_IMG_DATA --batch 32

/content/FcF-Inpainting/training/training_loop.py:397: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if (network_snapshot_ticks is not None) and (done or cur_tick % network_snapshot_ticks == 0) and cur_tick is not 0:
/content/FcF-Inpainting/training/training_loop.py:412: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if (snapshot_data is not None) and metrics and (done or cur_tick % network_snapshot_ticks == 0) and cur_tick is not 0:
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'

Training options:
{
  "num_gpus": 8,
  "eval_img_data": "/content/FcF-Inpainting/mydata/place2_val",
  "resolution": null,
  "image_snapshot_ticks": 50,
  "network_snapshot_ticks": 50,
  "metrics": true,
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.data.dataset.ImageDataset",
    "img_path": "/content/FcF-Inpainting/mydata/place2_train",
    "use_labels": false,
    "max_size": 1000,
    "xflip": false,
    "resolution": 256
  },
  "data_

In [6]:
!mkdir -p datasets/
!mkdir datasets/places2_dataset/


In [10]:
!wget http://data.csail.mit.edu/places/places365/train_large_places365challenge.tar
!tar -xvf train_large_places365challenge.tar -C datasets/places2_dataset/
!mv datasets/places2_datasets/data_large datasets/places2_dataset/train


串流輸出內容已截斷至最後 5000 行。
data_large/a/atrium/public/00013233.jpg
data_large/a/atrium/public/00001793.jpg
data_large/a/atrium/public/00013594.jpg
data_large/a/atrium/public/00001034.jpg
data_large/a/atrium/public/00015478.jpg
data_large/a/atrium/public/00010326.jpg
data_large/a/atrium/public/00002686.jpg
data_large/a/atrium/public/00010481.jpg
data_large/a/atrium/public/00002121.jpg
data_large/a/atrium/public/00006798.jpg
data_large/a/atrium/public/00011566.jpg
data_large/a/atrium/public/00011914.jpg
data_large/a/atrium/public/00014238.jpg
data_large/a/atrium/public/00003761.jpg
data_large/a/atrium/public/00003017.jpg
data_large/a/atrium/public/00006749.jpg
data_large/a/atrium/public/00011210.jpg
data_large/a/atrium/public/00018529.jpg
data_large/a/atrium/public/00007109.jpg
data_large/a/atrium/public/00002657.jpg
data_large/a/atrium/public/00019369.jpg
data_large/a/atrium/public/00010450.jpg
data_large/a/atrium/public/00010822.jpg
data_large/a/atrium/public/00001742.jpg
data_large/a/atrium

In [8]:
!wget http://data.csail.mit.edu/places/places365/val_large.tar
!tar -xvf val_large.tar -C datasets/places2_dataset/
!mv datasets/places2_dataset/val_large datasets/places2_dataset/val


串流輸出內容已截斷至最後 5000 行。
val_large/Places365_val_00026825.jpg
val_large/Places365_val_00023309.jpg
val_large/Places365_val_00014340.jpg
val_large/Places365_val_00018327.jpg
val_large/Places365_val_00014895.jpg
val_large/Places365_val_00003619.jpg
val_large/Places365_val_00034650.jpg
val_large/Places365_val_00006147.jpg
val_large/Places365_val_00018480.jpg
val_large/Places365_val_00019915.jpg
val_large/Places365_val_00019567.jpg
val_large/Places365_val_00015500.jpg
val_large/Places365_val_00015972.jpg
val_large/Places365_val_00022549.jpg
val_large/Places365_val_00027217.jpg
val_large/Places365_val_00007707.jpg
val_large/Places365_val_00035010.jpg
val_large/Places365_val_00002059.jpg
val_large/Places365_val_00028365.jpg
val_large/Places365_val_00016867.jpg
val_large/Places365_val_00016415.jpg
val_large/Places365_val_00024302.jpg
val_large/Places365_val_00004612.jpg
val_large/Places365_val_00036105.jpg
val_large/Places365_val_00008675.jpg
val_large/Places365_val_00025930.jpg
val_large/Places3

In [9]:
!bash tools/prepare_places_val.sh

random_masks: 100% 5000/5000 [00:00<00:00, 801418.53it/s]
...done
Preparing images...
Traceback (most recent call last):
  File "/content/FcF-Inpainting/tools/gen_masks.py", line 11, in <module>
    from training.data.gen_loader import get_loader
ModuleNotFoundError: No module named 'training'
